In [1]:
include("../src/HDC.jl")
include("../src/math.jl")
include("../src/experimental.jl")
using JLD

phalp_bow_esm = load("../data/phalp_bow_esm.jld")["embedded_bow_esm"]
phalp_bow_rand = load("../data/phalp_bow_rand.jld")["embedded_rand_bow"]
phalp_cnn_esm = load("../data/phalp_CNN_esm.jld")["embedded_CNN_esm"]
upi2doms = load("../data/phalp_upi2doms.jld")["upi2doms"]
domacc2domname = load("../data/phalp_domacc2domname.jld")["domacc2domname"]
phalp_cnn_rand = load("../data/phalp_cnn_rand.jld")["embedded_rand_CNN"]
non_ml = load("../data/phalp_non_ml.jld")["up_evd_type"]
ml_pred = load("../data/phalp_ml.jld")["up_evd_type_ML"]
up2type = load("../data/phalp_type.jld")["up2type"]


┌ Warning: type InlineStrings.String31 not present in workspace; reconstructing
└ @ JLD /home/mfat/.julia/packages/JLD/S6t6A/src/jld_types.jl:697


Dict{String, Union{Missing, String}} with 11549 entries:
  "UPI000178DD30" => "endolysin"
  "UPI0013EFDC93" => "endolysin"
  "UPI000172D062" => "VAL"
  "UPI001463ABBB" => "endolysin"
  "UPI000232F56D" => "endolysin"
  "UPI0011625D30" => "VAL"
  "UPI0009882324" => "endolysin"
  "UPI000CA1D611" => "VAL"
  "UPI0006BC2F8A" => "endolysin"
  "UPI000BBF7878" => "endolysin"
  "UPI00138B2696" => "endolysin"
  "UPI00025D6AED" => "endolysin"
  "UPI0010C2D3EE" => "endolysin"
  "UPI000D22144F" => "VAL"
  "UPI0018623B24" => "endolysin"
  "UPI00080F0655" => "endolysin"
  "UPI00022BD3A3" => "endolysin"
  "UPI0008093543" => "endolysin"
  "UPI001463E938" => "VAL"
  ⋮               => ⋮

In [2]:
new = filter(x -> (first(x) in collect(keys(non_ml))), up2type)
val = filter(x -> (last(x) == "VAL"), new)
endo = filter(x -> (last(x) == "endolysin"), new)

Dict{String, Union{Missing, String}} with 2803 entries:
  "UPI000178DD30" => "endolysin"
  "UPI0012B4B15F" => "endolysin"
  "UPI0013EFDC93" => "endolysin"
  "UPI000201BE9F" => "endolysin"
  "UPI000232F56D" => "endolysin"
  "UPI000DF0A1E1" => "endolysin"
  "UPI00138B2696" => "endolysin"
  "UPI000012EA4B" => "endolysin"
  "UPI000178C353" => "endolysin"
  "UPI001436E76F" => "endolysin"
  "UPI000F6BA7D8" => "endolysin"
  "UPI00001A38F6" => "endolysin"
  "UPI00001A9BAB" => "endolysin"
  "UPI001435C02C" => "endolysin"
  "UPI0010B96635" => "endolysin"
  "UPI0015F22725" => "endolysin"
  "UPI000D7DCFA8" => "endolysin"
  "UPI0010B8D78B" => "endolysin"
  "UPI000EB70E47" => "endolysin"
  ⋮               => ⋮

# Type classifier

## Purely hyperdimensional

Bad... convolutional embeddings of random hdvs seem to perform even worse weirdly enough

In [3]:
using MLDataPattern
using DataFrames

function type_classifier(embeddings)
    f1 = 0
    Threads.@threads for i in 1:50
        Keys = [key for (key, val) in new]
        key_seq = [embeddings[i] for i in Keys] # change mebeddings here
        Values = [val for (key, val) in new]

        (X_train, y_train), (X_test, y_test)  = stratifiedobs((key_seq, Values))

        train = DataFrame(hdv = X_train, type = y_train)
        test = DataFrame(hdv = X_test, type = y_test)

        val_hdv = bitadd(train[train.type .== "VAL", "hdv"]...)
        endo_hdv = bitadd(train[train.type .== "endolysin", "hdv"]...)

        confusion_matrix = zeros(2,2) # columns = real endo and real VAL, rows = predicted endo and predicted VAL

        Threads.@threads for i in 1:length(y_test)
            endo_sim = hamming(test.hdv[i], endo_hdv)
            val_sim = hamming(test.hdv[i], val_hdv)
            if endo_sim > val_sim
                if test.type[i] == "endolysin"
                    confusion_matrix[1,1] += 1
                else
                    confusion_matrix[2,1] += 1
                end
            else
                if test.type[i] == "VAL"
                    confusion_matrix[2,2] += 1
                else
                    confusion_matrix[1,2] += 1
                end
            end
        end

        prec = confusion_matrix[1,1]/(confusion_matrix[1,1] + confusion_matrix[1,2])
        recall = confusion_matrix[1,1]/(confusion_matrix[1,1] + confusion_matrix[2,1])

        f1 += 2 * (prec*recall)/(prec+recall)
    end

    return f1/50
end

type_classifier (generic function with 1 method)

In [4]:
println(type_classifier(phalp_bow_rand))
println(type_classifier(phalp_bow_esm))
println(type_classifier(phalp_cnn_rand))
println(type_classifier(phalp_cnn_esm))

0.13902353532694728


0.1525937736396466


0.09598247702826664


0.14971362988002368


## XGBoost with hdv embeddings

In [51]:
using MLJ
using MLDataPattern
using DataFrames
using MLJXGBoostInterface

function xgbooster(embeddings)
    Keys = [key for (key, val) in new]
    key_seq = nested_arrays2mat([embeddings[i] for i in Keys], true) # change emebeddings here
    Values = [val for (key, val) in new]
    Values = [i == "VAL" ? 0 : 1 for i in Values]

    data = DataFrame(Int.(key_seq), :auto)
    data = insertcols!(data, 1, :type => Values) 

    y, X = unpack(data, ==(:type))

    train, test = partition(eachindex(y), 0.8, shuffle=true)
    Tree = MLJ.@load XGBoostClassifier
    tree = Tree()
    mach = machine(tree, X, y, scitype_check_level=0)


end

xgbooster(phalp_bow_rand)

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/mfat/.julia/packages/MLJModels/OJDDo/src/loading.jl:159


┌ Warning: The number and/or types of data arguments do not match what the specified model
│ supports. Suppress this type check by specifying `scitype_check_level=0`.
│ 
│ Run `@doc DecisionTree.DecisionTreeClassifier` to learn more about your model's requirements.
│ 
│ Commonly, but non exclusively, supervised models are constructed using the syntax
│ `machine(model, X, y)` or `machine(model, X, y, w)` while most other models are
│ constructed with `machine(model, X)`.  Here `X` are features, `y` a target, and `w`
│ sample or class weights.
│ 
│ In general, data in `machine(model, data...)` is expected to satisfy
│ 
│     scitype(data) <: MLJ.fit_data_scitype(model)
│ 
│ In the present case:
│ 
│ scitype(data) = Tuple{Table{AbstractVector{Count}}, AbstractVector{Count}}
│ 
│ fit_data_scitype(model) = Tuple{Table{<:Union{AbstractVector{<:Continuous}, AbstractVector{<:Count}, AbstractVector{<:OrderedFactor}}}, AbstractVector{<:Finite}}
└ @ MLJBase /home/mfat/.julia/packages/MLJBase/WKVE

untrained Machine; caches model-specific representations of data
  model: DecisionTreeClassifier(max_depth = -1, …)
  args: 
    1:	Source @813 ⏎ Table{AbstractVector{Count}}
    2:	Source @979 ⏎ AbstractVector{Count}


In [46]:
Keys = [key for (key, val) in new]
key_seq = nested_arrays2mat([phalp_bow_rand[i] for i in Keys], true) # change emebeddings here
Values = [val for (key, val) in new]
data = DataFrame(Int.(key_seq), :auto)
data = insertcols!(data, 1, :type => Values)

y, X = unpack(data, ==(:type), colname -> true)

MLJ.schema(data)


┌───────┬──────────┬────────┐
│ names │ scitypes │ types  │
├───────┼──────────┼────────┤
│ type  │ Textual  │ String │
│ x1    │ Count    │ Int64  │
│ x2    │ Count    │ Int64  │
│ x3    │ Count    │ Int64  │
│ x4    │ Count    │ Int64  │
│ x5    │ Count    │ Int64  │
│ x6    │ Count    │ Int64  │
│ x7    │ Count    │ Int64  │
│ x8    │ Count    │ Int64  │
│ x9    │ Count    │ Int64  │
│ x10   │ Count    │ Int64  │
│ x11   │ Count    │ Int64  │
│ x12   │ Count    │ Int64  │
│ x13   │ Count    │ Int64  │
│ x14   │ Count    │ Int64  │
│ x15   │ Count    │ Int64  │
│   ⋮   │    ⋮     │   ⋮    │
└───────┴──────────┴────────┘
            9985 rows omitted


# Domain classifier